In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
# roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,8)

# # 青海省部分地区
roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# 图像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

In [ ]:
# 加载数据
l8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31')\
    .map(water_index)
landsat_images = l8_images.map(maskL8sr)
Map.addLayer(landsat_images.mosaic(),visParams,'landsat image')
print(landsat_images.size().getInfo())

In [ ]:
print(l8_images.first().getInfo())

# 生成样本集

## 筛选得出采样影像

In [ ]:
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14,
})

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(ImageArea).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(7)))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

##  定义类区

In [ ]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# # Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
# Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
# Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
# Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# # 三者的交集
# permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
# landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
# seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
# 两者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

In [ ]:
# 定义区间均值函数
def intervalmean(image):
    awei = image.updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': image.geometry(), 
        'scale': 30, 
        'bestEffort': True
    })
    return image.set({'mean': intervalMean.get('AWEI')})

In [ ]:
a = sample_image.limit(39).map(intervalmean)
b = a.aggregate_array('mean')
print(b.getInfo())

## 图像采样函数

In [ ]:
# 设置采样个数
number = 500
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))
    # 陆地,此处用image.geometry()，是为了减少“陆地”样本
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(image.get('mean')))
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points

## 基于“采样图像”，生成样本集，是否下载样本数据

In [ ]:
# 总的样本点数
points_collection = a.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))

# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# # 非水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
### 是否下载样本集数据（下载到Google Drive上）
# label = 'qinghai_2018'
# # csv格式
# selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','ndvi_mndwi','waterclass']
# geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='csv', selectors=selector)

# # shp格式
# selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','ndvi_mndwi','waterclass']
# geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='shp', selectors=selector)

#  使用Random forest,训练样本集

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'
# 生成RF训练器
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

#  图像校正函数

In [ ]:
def AutomaticCorrection(img):
    image = maskL8sr(img).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    basemap = ee.Image.constant(0).toFloat().clip(img.geometry()).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': img.geometry(),
        'scale': 30,
        'bestEffort': True
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water.set({'occurrence_threshold':occurrence_threshold})

#  加载研究区内的某一景影像

##  未校正的RF的分类影像

In [ ]:
# 添加一个底图白板
# image9 = water_index(maskL8sr(ee.Image(landsat_images.toList(landsat_images.size()).get(5))))
image9 = water_index(maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627')))
Map.addLayer(image9.clip(roi),visParams,'image9')

basemap = ee.Image.constant(0).toFloat().clip(image9.geometry()).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

In [ ]:
classify_image = image9.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
# Map.addLayer(classify_image.selfMask(),{'palette':['green']}, 'classify image')
Map.addLayer(classify_image,{'palette':['red','blue'],'min':0,'max':1}, 'classify image')

## 将上一步的分类影像进行校正

In [ ]:
a = AutomaticCorrection(classify_image)
Map.addLayer(a,{'palette':['white','red'],'min':0,'max':1}, 'enhanced_water')

# 制图

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
raw_image = maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627').clip(roi))
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(landsat_images.mosaic().clip(raw_image.geometry()), region=region, vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'reference image', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(enhanced_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'classified image(AutomaticCorrection)', fontsize=20)